# SIDA: Speaker Identification for Archives

In [10]:
import attk
import os
import csv
import numpy as np
import librosa
import timeit
import random
import subprocess
import urllib2
from sklearn.externals import joblib
from numpy import ma
from aubio import source, pitch
from moviepy.audio.io import AudioFileClip
from IPython.display import display, Audio
import pandas as pd

!mkdir -p /sharedfolder/sida_classifier/

os.chdir('/sharedfolder/sida_classifier/')

training_audio_dir_name = "Bill_Clinton"

In [11]:
## Load 1-second labels

csv_path = '/sharedfolder/sida_classifier/Bill_Clinton_5K_labels.csv'

train_table_df = pd.read_csv(csv_path, names=['Media file basename', 'Start', 'Duration', 'Label'])

train_table_df.head()

,Media file basename,Start,Duration,Label
0,CSPAN3_20160207_061500_Bill_Clinton_1992_New_H...,36.0,1.0,Bill Clinton
1,CNNW_20150614_130000_State_of_the_Union_With_J...,1.0,1.0,Not Bill Clinton
2,CNNW_20150614_130000_State_of_the_Union_With_J...,1.0,1.0,Not Bill Clinton
3,CNNW_20150614_130000_State_of_the_Union_With_J...,1.0,1.0,Not Bill Clinton
4,CNNW_20150614_130000_State_of_the_Union_With_J...,1.0,1.0,Not Bill Clinton


In [14]:
## Choosing variables to extract and assigning variables we'll use below

labels_to_use = ['Bill Clinton']

media_dir_pathname = '/sharedfolder/sida_classifier/' + training_audio_dir_name

class_dir_pathname = '/sharedfolder/sida_classifier/_classes_' + training_audio_dir_name

In [15]:
%%capture
## Excerpting WAV clips corresponding to labels
#### (This may take a while.)

os.chdir('/sharedfolder/sida_classifier/')

train_table_groups = train_table_df.groupby(['Media file basename', 'Label'])

for name, group in train_table_groups:
    list_of_lists = group.values.tolist()
    basename, start, duration, label = list_of_lists[0]
    filename = str(basename) + '.wav'
    media_path = os.path.join(media_dir_pathname, filename)
    if not os.path.isfile(media_path):
        filename = str(basename) + '.mp3'
        media_path = os.path.join(media_dir_pathname, filename)
    label_dir_pathname = str(os.path.join(class_dir_pathname, label.replace(' ','_')))
    if label in labels_to_use:
        subclip_pairs = []
        for row in list_of_lists:
            basename, start, duration, label = row
            subclip_pairs.append((float(start), float(duration)))
        try: subprocess.call(['mkdir', '-p', label_dir_pathname])
        except: pass
        try:
            attk.subclip_list(media_path, subclip_pairs, label_dir_pathname)
        except Exception as e: 
            print(e)

In [24]:
%%capture
## Extract vowel segments from labeled audio clips
#### (This may take a while.)

os.chdir(class_dir_pathname)

for dir_name in [item for item in os.listdir('./') if os.path.isdir(item)]:
    try:
        attk.batch_extract_vowels(dir_name)
    except Exception as e:
        print("ERROR: " + dir_name)
        print(e)
    os.chdir(class_dir_pathname)

In [28]:
## %%capture allows you to skip error comments - marking it out for now
## Extract features (MFCCs, deltas, and delta-deltas) from all clips, then write features to CSVs

os.chdir(class_dir_pathname)

for dir_name in [item for item in os.listdir('./') if os.path.isdir(item)]:
    print("> Starting " + dir_name)
    try:
        os.chdir(os.path.join(class_dir_pathname, dir_name, '_vowel_clips'))
        try: os.mkdir('../_vowel_mfccs_and_deltas')
        except: pass
        filenames = [item for item in os.listdir('./') if item[-4:].lower()=='.wav']
        for filename in filenames:
            csv_out_path = '../_vowel_mfccs_and_deltas/' + filename[:-4] + '.mfcc.csv'
            if not os.path.isfile(csv_out_path):
                try:
                    mfccs = attk.get_mfccs_and_deltas(filename, n_mfcc=30, n_fft=4096, freq_min=100, freq_max=16000)
                    if len(mfccs) > 0:
                        with open(csv_out_path, 'w') as fo:
                            csv_writer = csv.writer(fo)
                            csv_writer.writerows(mfccs)  
                except Exception as e:
                    print('FILE ERROR: ' + filename)
                    print(e)
    except Exception as e:
        print('SKIPPING DIRECTORY: ' + dir_name)     ## Skipping class directories for which we didn't extract vowels
        #print(e)

> Starting males_5k
> Starting Bill_Clinton
FILE ERROR: cpb-aacip-503-6688g8g18r__NHPR95189__252.0_7.0__1.0_1.0__0.8359_0.0929.wav
when mode='interp', width=9 cannot exceed data.shape[axis]=2
FILE ERROR: cpb-aacip-298-89d51qxn.h264__183.3796_92.0__50.0_1.0__0.4644_0.1858.wav
when mode='interp', width=9 cannot exceed data.shape[axis]=2
FILE ERROR: cpb-aacip-503-s17sn01t72__NHPR95200__141.96_203.0__145.0_1.0__0.5573_0.0929.wav
when mode='interp', width=9 cannot exceed data.shape[axis]=2
FILE ERROR: cpb-aacip-503-j96057dh5p__NHPR95198__337.0027_89.9__59.0_1.0__0.0929_0.1858.wav
when mode='interp', width=9 cannot exceed data.shape[axis]=2
FILE ERROR: cpb-aacip-503-s17sn01t72__NHPR95200__3472.92_94.28__67.0_1.0__0.4644_0.2786.wav
when mode='interp', width=9 cannot exceed data.shape[axis]=4
FILE ERROR: cpb-aacip-111-945qg6dq.h264__923.0_43.0__37.0_1.0__0.0929_0.5573.wav
when mode='interp', width=9 cannot exceed data.shape[axis]=7
FILE ERROR: cpb-aacip-111-9995xkn2.h264__849.0_8.0__0.0_1.0__0

FILE ERROR: cpb-aacip-43-54xgxppn__520.5_88.36__71.0_1.0__0.8359_0.1641.wav
when mode='interp', width=9 cannot exceed data.shape[axis]=2
FILE ERROR: cpb-aacip-43-54xgxppn__1726.0373_138.0__87.0_1.0__0.0_0.3715.wav
when mode='interp', width=9 cannot exceed data.shape[axis]=5
FILE ERROR: cpb-aacip-503-6688g8g18r__NHPR95189__269.0_51.0__43.0_1.0__0.8359_0.0929.wav
when mode='interp', width=9 cannot exceed data.shape[axis]=2
FILE ERROR: cpb-aacip-43-54xgxppn__520.5_88.36__64.0_1.0__0.0929_0.0929.wav
when mode='interp', width=9 cannot exceed data.shape[axis]=2
FILE ERROR: cpb-aacip-503-s17sn01t72__NHPR95200__352.128_74.0__40.0_1.0__0.5573_0.0929.wav
when mode='interp', width=9 cannot exceed data.shape[axis]=2
FILE ERROR: cpb-aacip-298-89d51qxn.h264__184.0_86.0__19.0_1.0__0.0929_0.1858.wav
when mode='interp', width=9 cannot exceed data.shape[axis]=2
FILE ERROR: cpb-aacip-503-3b5w669n0k__NHPR95188__1534.0_265.0__116.0_1.0__0.8359_0.1641.wav
when mode='interp', width=9 cannot exceed data.shape

FILE ERROR: cpb-aacip-111-80vq8dc6.h264__247.0_142.0__45.0_1.0__0.0_0.3715.wav
when mode='interp', width=9 cannot exceed data.shape[axis]=5
FILE ERROR: cpb-aacip-503-3b5w669n0k__NHPR95188__1811.04_363.0__108.0_1.0__0.2786_0.0929.wav
when mode='interp', width=9 cannot exceed data.shape[axis]=2
FILE ERROR: cpb-aacip-503-s17sn01t72__NHPR95200__2440.192_148.8213__143.0_1.0__0.6502_0.3498.wav
when mode='interp', width=9 cannot exceed data.shape[axis]=4
FILE ERROR: cpb-aacip-503-6688g8g18r__NHPR95189__930.4747_110.0__22.0_1.0__0.9288_0.0712.wav
when mode='interp', width=9 cannot exceed data.shape[axis]=1
FILE ERROR: cpb-aacip-293-97kps24q.h264__1389.0_632.0__543.0_1.0__0.8359_0.0929.wav
when mode='interp', width=9 cannot exceed data.shape[axis]=2
FILE ERROR: cpb-aacip-293-97kps24q.h264__1287.0531_828.0__717.0_1.0__0.0929_0.3715.wav
when mode='interp', width=9 cannot exceed data.shape[axis]=5
FILE ERROR: cpb-aacip-43-54xgxppn__360.0_46.0__30.0_1.0__0.2786_0.1858.wav
when mode='interp', width=

FILE ERROR: cpb-aacip-189-12m6402b.h264__31.5791_20.6__12.0_1.0__0.0929_0.5573.wav
when mode='interp', width=9 cannot exceed data.shape[axis]=7
FILE ERROR: cpb-aacip-293-b27pn8xp9q__HUT00000107_.h264__3371.0_112.0__32.0_1.0__0.5573_0.3715.wav
when mode='interp', width=9 cannot exceed data.shape[axis]=4
FILE ERROR: cpb-aacip-189-12m6402b.h264__31.5791_20.6__12.0_1.0__0.743_0.257.wav
when mode='interp', width=9 cannot exceed data.shape[axis]=3
FILE ERROR: cpb-aacip-503-tx3513vq37__NHPR95203__170.0_8.0__2.0_1.0__0.0_0.0929.wav
when mode='interp', width=9 cannot exceed data.shape[axis]=2
FILE ERROR: cpb-aacip-503-6688g8g18r__NHPR95189__149.0_86.0__23.0_1.0__0.1858_0.1858.wav
when mode='interp', width=9 cannot exceed data.shape[axis]=3
FILE ERROR: cpb-aacip-503-s17sn01t72__NHPR95200__436.0_6.0__0.0_1.0__0.0_0.1858.wav
when mode='interp', width=9 cannot exceed data.shape[axis]=3
FILE ERROR: cpb-aacip-503-6688g8g18r__NHPR95189__1637.5467_56.0__37.0_1.0__0.1858_0.2786.wav
when mode='interp', w

FILE ERROR: cpb-aacip-503-6688g8g18r__NHPR95189__1319.0_15.0__1.0_1.0__0.0_0.2786.wav
when mode='interp', width=9 cannot exceed data.shape[axis]=4
FILE ERROR: cpb-aacip-111-9995xkn2.h264__393.0_21.0__4.0_1.0__0.5573_0.1858.wav
when mode='interp', width=9 cannot exceed data.shape[axis]=3
FILE ERROR: cpb-aacip-298-89d51qxn.h264__874.0107_205.0__173.0_1.0__0.2786_0.0929.wav
when mode='interp', width=9 cannot exceed data.shape[axis]=2
FILE ERROR: cpb-aacip-111-51vdnn24.h264__400.0_38.0__28.0_1.0__0.0929_0.2786.wav
when mode='interp', width=9 cannot exceed data.shape[axis]=4
FILE ERROR: cpb-aacip-298-89d51qxn.h264__541.0_256.0__221.0_1.0__0.0929_0.0929.wav
when mode='interp', width=9 cannot exceed data.shape[axis]=2
FILE ERROR: cpb-aacip-503-3b5w669n0k__NHPR95188__1534.0_265.0__5.0_1.0__0.8359_0.0929.wav
when mode='interp', width=9 cannot exceed data.shape[axis]=2
FILE ERROR: cpb-aacip-503-tx3513vq37__NHPR95203__195.36_100.0__38.0_1.0__0.2786_0.0929.wav
when mode='interp', width=9 cannot ex

...

FILE ERROR: cpb-aacip-259-kw57hn2v.h264__75.0_85.0__0.0_1.0__0.5573_0.2786.wav
when mode='interp', width=9 cannot exceed data.shape[axis]=4
FILE ERROR: cpb-aacip-17-848pm1fs.h264__1059.0_1069.0__0.0_1.0__0.5573_0.2786.wav
when mode='interp', width=9 cannot exceed data.shape[axis]=4
FILE ERROR: cpb-aacip-15-g73707wx0d__21926-2009_10_19_.h264__1053.0_1063.0__0.0_1.0__0.8359_0.1641.wav
when mode='interp', width=9 cannot exceed data.shape[axis]=2
FILE ERROR: cpb-aacip-43-kw57d2qp2j__10354812___20.932_30.932__1.0_2.0__0.2786_0.0929.wav
when mode='interp', width=9 cannot exceed data.shape[axis]=2
FILE ERROR: cpb-aacip-15-gx44q7qz61__27537-2011_02_15CAP14-RAW_.h264__28.0_38.0__1.0_2.0__0.1858_0.1858.wav
when mode='interp', width=9 cannot exceed data.shape[axis]=3
FILE ERROR: cpb-aacip-15-bn9x05xf50__phy03_vid_zkazoo_.h264__20.0_30.0__1.0_2.0__0.0_0.0929.wav
when mode='interp', width=9 cannot exceed data.shape[axis]=2
FILE ERROR: cpb-aacip-129-053ffgdb.h264__405.0_415.0__1.0_2.0__0.6502_0.3498

FILE ERROR: cpb-aacip-293-03qv9sj9.h264__869.0_879.0__3.0_4.0__0.8359_0.1641.wav
when mode='interp', width=9 cannot exceed data.shape[axis]=2
FILE ERROR: cpb-aacip-15-84mkmc9h.h264__2107.0_2117.0__0.0_1.0__0.0_0.4644.wav
when mode='interp', width=9 cannot exceed data.shape[axis]=6
FILE ERROR: cpb-aacip-77-82k6fr9p.h264__701.0_711.0__4.0_5.0__0.9288_0.0712.wav
when mode='interp', width=9 cannot exceed data.shape[axis]=1
FILE ERROR: cpb-aacip-35-06g1jzc4.h264__2319.0_2329.0__0.0_1.0__0.0_0.2786.wav
when mode='interp', width=9 cannot exceed data.shape[axis]=4
FILE ERROR: cpb-aacip-15-513tt4fv0v__orlando_hour_2_jackie_class_.h264__38.0_48.0__1.0_2.0__0.0_0.1858.wav
when mode='interp', width=9 cannot exceed data.shape[axis]=3
FILE ERROR: cpb-aacip-17-81wdcsbg.h264__1571.0_1581.0__2.0_3.0__0.6502_0.1858.wav
when mode='interp', width=9 cannot exceed data.shape[axis]=3
FILE ERROR: cpb-aacip-394-601zd472.h264__1587.0_1597.0__1.0_2.0__0.6502_0.3498.wav
when mode='interp', width=9 cannot exceed d

FILE ERROR: cpb-aacip-80-7312kdpj__6135.0_6145.0__0.0_1.0__0.0_0.1858.wav
when mode='interp', width=9 cannot exceed data.shape[axis]=3
FILE ERROR: cpb-aacip-43-70msc097__343.0_353.0__2.0_3.0__0.743_0.257.wav
when mode='interp', width=9 cannot exceed data.shape[axis]=3
FILE ERROR: cpb-aacip-259-b27prp2c.h264__497.0_507.0__3.0_4.0__0.743_0.0929.wav
when mode='interp', width=9 cannot exceed data.shape[axis]=2
FILE ERROR: cpb-aacip-500-th8bmm92__953.0_963.0__2.0_3.0__0.1858_0.1858.wav
when mode='interp', width=9 cannot exceed data.shape[axis]=3
FILE ERROR: cpb-aacip-77-11kh29nm.h264__641.0_651.0__4.0_5.0__0.6502_0.2786.wav
when mode='interp', width=9 cannot exceed data.shape[axis]=3
FILE ERROR: cpb-aacip-16-gf0ms3kd8b__focus040227b___2130.0_2140.0__0.0_1.0__0.0_0.2786.wav
when mode='interp', width=9 cannot exceed data.shape[axis]=4
FILE ERROR: cpb-aacip-55-6341pnc9.h264__770.0_780.0__1.0_2.0__0.9288_0.0712.wav
when mode='interp', width=9 cannot exceed data.shape[axis]=1
FILE ERROR: cpb-aac

FILE ERROR: cpb-aacip-309-117m0dvj__Boss-1__39.0_49.0__4.0_5.0__0.0_0.1858.wav
when mode='interp', width=9 cannot exceed data.shape[axis]=3
FILE ERROR: cpb-aacip-43-70msc097__343.0_353.0__3.0_4.0__0.0_0.0929.wav
when mode='interp', width=9 cannot exceed data.shape[axis]=2
FILE ERROR: cpb-aacip-398-870vtds8.h264__1708.0_1718.0__3.0_4.0__0.6502_0.0929.wav
when mode='interp', width=9 cannot exceed data.shape[axis]=2
FILE ERROR: cpb-aacip-50-741rnjc9__896.0_906.0__4.0_5.0__0.6502_0.1858.wav
when mode='interp', width=9 cannot exceed data.shape[axis]=3
FILE ERROR: cpb-aacip-15-nz80k26p0q__BTP_20091030_weekly_.h264__646.0_656.0__4.0_5.0__0.0_0.6502.wav
when mode='interp', width=9 cannot exceed data.shape[axis]=8
FILE ERROR: cpb-aacip-15-vx05x25r0r__27537-2011_02_15_.h264__356.0_366.0__4.0_5.0__0.5573_0.1858.wav
when mode='interp', width=9 cannot exceed data.shape[axis]=3
FILE ERROR: cpb-aacip-394-601zd472.h264__1276.0_1286.0__4.0_5.0__0.5573_0.4427.wav
when mode='interp', width=9 cannot excee

FILE ERROR: cpb-aacip-17-69867tp8.h264__506.0_516.0__0.0_1.0__0.743_0.257.wav
when mode='interp', width=9 cannot exceed data.shape[axis]=3
FILE ERROR: cpb-aacip-43-7312k28f__1214.0_1224.0__4.0_5.0__0.0_0.3715.wav
when mode='interp', width=9 cannot exceed data.shape[axis]=5
FILE ERROR: cpb-aacip-500-j678xk7n__1380.0_1390.0__4.0_5.0__0.6502_0.0929.wav
when mode='interp', width=9 cannot exceed data.shape[axis]=2
FILE ERROR: cpb-aacip-43-70msc097__343.0_353.0__2.0_3.0__0.5573_0.0929.wav
when mode='interp', width=9 cannot exceed data.shape[axis]=2
FILE ERROR: cpb-aacip-15-qn5z60c79t__26361-2010_11_16_.h264__2819.0_2829.0__4.0_5.0__0.6502_0.2786.wav
when mode='interp', width=9 cannot exceed data.shape[axis]=3
FILE ERROR: cpb-aacip-43-7312k28f__1214.0_1224.0__4.0_5.0__0.9288_0.0712.wav
when mode='interp', width=9 cannot exceed data.shape[axis]=1
FILE ERROR: cpb-aacip-46-311ns5br.h264__564.0_574.0__1.0_2.0__0.0_0.0929.wav
when mode='interp', width=9 cannot exceed data.shape[axis]=2
FILE ERROR:

In [ ]:
## Continue to the next notebook to train and run the speaker ID classifier.